# Parallel Optimization with APOSMM

This tutorial demonstrates libEnsemble’s capability to identify multiple minima of simulation output using the built-in APOSMM (Asynchronously Parallel Optimization Solver for finding Multiple Minima) ``gen_f``. In this tutorial, we’ll create a simple simulation ``sim_f`` that defines a function with multiple minima, then write a libEnsemble calling script that imports APOSMM and parameterizes it to check for minima over a domain of outputs from our ``sim_f``.

## Six-Hump Camel Simulation Function

Describing APOSMM’s operations is simpler with a given function on which to depict evaluations. We’ll use the Six-Hump Camel function, known to have six global minima. A sample space of this function, containing all minima, appears below:

![6humpcamel](images/basic_6hc.png)

*Note: The following ``sim_f`` won't operate stand-alone since it has not yet been parameterized and called by libEnsemble. The full routine should work as expected.*

Create a new Python file named ``six_hump_camel.py``. This will be our ``sim_f``, incorporating the above function. Write the following:

In [2]:
import numpy as np

def six_hump_camel(H, persis_info, sim_specs, _):
    """Six-Hump Camel sim_f."""

    batch = len(H['x'])                            # Num evaluations each sim_f call.
    H_o = np.zeros(batch, dtype=sim_specs['out'])  # Define output array H

    for i, x in enumerate(H['x']):
        H_o['f'][i] = three_hump_camel_func(x)     # Function evaluations placed into H

    return H_o, persis_info


def six_hump_camel_func(x):
    """ Six-Hump Camel function definition """
    x1 = x[0]
    x2 = x[1]
    term1 = (4-2.1*x1**2+(x1**4)/3) * x1**2
    term2 = x1*x2
    term3 = (-4+4*x2**2) * x2**2

    return term1 + term2 + term3

## APOSMM Operations

APOSMM coordinates multiple local optimization runs starting from a collection of sample points. These local optimization runs occur in parallel, and can incorporate a variety of optimization methods, including from NLopt, PETSc/TAO, and SciPy. Some number of uniformly sampled points is returned by APOSMM for simulation evaluations before local optimization runs can occur, if no prior simulation evaluations are provided. User-requested sample points can also be provided to APOSMM:

![6hcsampling](images/sampling_6hc.png)

Specifically, APOSMM will begin local optimization runs from those points that don’t have better (more minimal) points nearby within a threshold ``r_k``. For the above example, after APOSMM has returned the uniformly sampled points, for simulation evaluations it will likely begin local optimization runs from the user-requested approximate minima. Providing these isn’t required, but can offer performance benefits.

Each local optimization run chooses new points and determines if they’re better by passing them back to be evaluated by the simulation routine. If so, new local optimization runs are started from those points. This continues until each run converges to a minimum:

![6hclocalopt](images/localopt_6hc.png)

Throughout, generated and evaluated points are appended to the History array, with the field ``'local_pt'`` being ``True`` if the point is part of a local optimization run, and ``'local_min'`` being ``True`` if the point has been ruled a local minimum.

## APOSMM Persistence

The most recent version of APOSMM included with libEnsemble is referred to as Persistent APOSMM. Unlike most other user functions that are initiated and completed by workers multiple times based on allocation, a single worker process initiates APOSMM so that it “persists” and keeps running over the course of the entire libEnsemble routine. APOSMM begins it’s own parallel evaluations and communicates points back and forth with the manager, which are then given to workers and evaluated by simulation routines.

In practice, since a single worker becomes “persistent” for APOSMM, users must ensure that enough workers or MPI ranks are initiated to support libEnsemble’s manager, a persistent worker to run APOSMM, and simulation routines. The following:

    mpiexec -n 3 python my_aposmm_routine.py
    
results in only one worker process available to perform simulation routines.

## Calling Script

Create a new Python file named ``my_first_aposmm.py``. Start by importing NumPy, libEnsemble routines, APOSMM, our ``sim_f``, and a specialized allocation function:

In [3]:
import numpy as np

from six_hump_camel import six_hump_camel

from libensemble.libE import libE
from libensemble.gen_funcs.persistent_aposmm import aposmm
from libensemble.alloc_funcs.persistent_aposmm_alloc import persistent_aposmm_alloc
from libensemble.tools import parse_args, add_unique_random_streams

ModuleNotFoundError: No module named 'six_hump_camel'

This allocation function starts a single Persistent APOSMM routine and provides ``sim_f`` output for points requested by APOSMM. Points can be sampled points or points from local optimization runs.

APOSMM supports a wide variety of external optimizers. The following statements set optimizer settings to ``'scipy'`` to indicate to APOSMM which optimization method to use, and help prevent unnecessary imports or package installations: